In [19]:
import json
import networkx as nx
import plotly.graph_objects as go

In [20]:
LEBL_TAXIWAYS = '/home/santi/Documents/GitHub-3º/AIrport/bluesky/resources/navdata/airports/LEBL/taxiways.json'

In [ ]:
class TaxiwayGraph:
    def __init__(
            self,
            file_path,
            configuration='West'
    ):
        self.file_path = file_path
        self.configuration = configuration
        self.G = nx.DiGraph()
        self.load_data()
        self.create_graph()
    
    def load_data(self):
        """
        Data loading
        """
        with open(self.file_path, 'r') as file:
            self.taxiways_data = json.load(file)
    
    def create_graph(self):
        """
        Builds the directed graph from taxiway data.
        """
        taxiways = self.taxiways_data["configurations"][self.configuration]["taxiways"]
        for taxiway in taxiways:
            name = taxiway["name"]
            lat, lon = taxiway["coordinates"]
            # Invert coordinates to not have an
            # specular image
            self.G.add_node(name, pos=(lon, lat))
            for connection in taxiway["connections"]:
                self.G.add_edge(name, connection)
    
    def visualize_graph(self):
        """
        Visualizes the directed graph using Plotly.
        """
        pos = nx.get_node_attributes(self.G, 'pos')
        edge_x = []
        edge_y = []
        for edge in self.G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
        
        edge_trace = go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=1, color="gray"),
            hoverinfo="none",
            mode="lines"
        )
        
        node_x = [pos[node][0] for node in self.G.nodes()]
        node_y = [pos[node][1] for node in self.G.nodes()]
        node_text = list(self.G.nodes())
        
        node_trace = go.Scatter(
            x=node_x, y=node_y,
            mode="markers+text",
            marker=dict(size=10, color="blue"),
            text=node_text,
            textposition="top center",
            hoverinfo="text"
        )
        
        arrow_annotations = [
            dict(
                ax=pos[edge[0]][0],
                ay=pos[edge[0]][1],
                axref='x',
                ayref='y',
                x=pos[edge[1]][0],
                y=pos[edge[1]][1],
                xref='x',
                yref='y',
                showarrow=True,
                arrowhead=2,
                arrowsize=1.5,
                arrowwidth=1.5,
                arrowcolor='gray'
            ) for edge in self.G.edges()
        ]
        
        fig = go.Figure(data=[edge_trace, node_trace])
        fig.update_layout(
            title="Directed Taxiways Graph",
            showlegend=False,
            hovermode="closest",
            margin=dict(b=20, l=5, r=5, t=40),
            xaxis=dict(title="Longitude", showgrid=False, zeroline=False),
            yaxis=dict(title="Latitude", showgrid=False, zeroline=False, scaleanchor="x"),
            annotations=arrow_annotations
        )
        fig.show()
    



## West configuration

In [22]:
graph = TaxiwayGraph(LEBL_TAXIWAYS)
graph.visualize_graph()

## East configuration

In [23]:
graph = TaxiwayGraph(LEBL_TAXIWAYS, configuration='East')
graph.visualize_graph()